# Target-Specific Bias Correction (exp_063)

**Hypothesis**: The bias is target-specific, not uniform. Applying different corrections to each target should improve ALL targets.

**Key Insight from exp_062**:
- Product 2: predictions are 0.009 LOWER than actuals (negative bias) -> ADD 0.009
- Product 3: predictions are 0.003 LOWER than actuals (negative bias) -> ADD 0.003
- SM: predictions are 0.021 HIGHER than actuals (positive bias) -> SUBTRACT 0.021

**Implementation**:
1. Use the best GP + MLP + LGBM ensemble (exp_032)
2. Apply target-specific corrections: `corrected_pred = ensemble_pred + corrections`
3. Where corrections = [+0.009, +0.003, -0.021] for [Product 2, Product 3, SM]

**Why this might work**:
- The bias is target-specific, not uniform
- Applying target-specific corrections should improve ALL targets
- This is a post-processing step that doesn't change the model

**Best baseline**: exp_032 CV 0.008194 (GP 0.15 + MLP 0.55 + LGBM 0.30)

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = (X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"])
        yield (X[~mask], Y[~mask]), (X[mask], Y[mask])

print('Data loading functions defined')

In [ ]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

In [ ]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip), dtype=torch.double)

# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')
print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

In [ ]:
# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLP model defined')

In [ ]:
# GP + MLP + LGBM Ensemble Model with Target-Specific Bias Correction
class TargetSpecificBiasEnsemble:
    """GP + MLP + LGBM ensemble with target-specific bias correction."""
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.30, 
                 corrections=None):
        self.data = data
        self.mixed = (data == 'full')
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        # Target-specific corrections: [Product 2, Product 3, SM]
        self.corrections = corrections if corrections is not None else np.array([0.0, 0.0, 0.0])
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        # Models
        self.gp_models = None
        self.mlp_model = None
        self.lgbm_models = None
        self.scaler_full = None
        self.scaler_simple = None
        
    def train_model(self, X, Y):
        # Prepare features
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        # Data augmentation for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            X_full_all = np.vstack([X_full, X_full_flip])
            X_simple_all = np.vstack([X_simple, X_simple_flip])
            Y_all = np.vstack([Y_np, Y_np])
        else:
            X_full_all = X_full
            X_simple_all = X_simple
            Y_all = Y_np
        
        # Scale features
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        X_full_scaled = self.scaler_full.fit_transform(X_full_all)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple_all)
        
        # Train GP (3 models, one per target)
        self.gp_models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42+i)
            gp.fit(X_simple_scaled, Y_all[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        self.mlp_model = MLPModelInternal(input_dim=X_full_scaled.shape[1], hidden_dims=[32, 16], dropout=0.05).double().to(device)
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=5e-4, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
        loss_fn = nn.HuberLoss()
        
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_all, dtype=torch.double).to(device)
        dataset = TensorDataset(X_tensor, Y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        self.mlp_model.train()
        for epoch in range(200):
            epoch_loss = 0
            for xb, yb in loader:
                optimizer.zero_grad()
                pred = self.mlp_model(xb)
                loss = loss_fn(pred, yb)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            scheduler.step(epoch_loss)
        self.mlp_model.eval()
        
        # Train LGBM (3 models, one per target)
        self.lgbm_models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        for i in range(3):
            train_data = lgb.Dataset(X_full_scaled, label=Y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=100)
            self.lgbm_models.append(model)
        
    def predict(self, X):
        # Prepare features
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
        
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        if self.mixed:
            X_full_flip_scaled = self.scaler_full.transform(X_full_flip)
            X_simple_flip_scaled = self.scaler_simple.transform(X_simple_flip)
        
        # GP predictions
        gp_pred = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gp_models])
        if self.mixed:
            gp_pred_flip = np.column_stack([gp.predict(X_simple_flip_scaled) for gp in self.gp_models])
            gp_pred = (gp_pred + gp_pred_flip) / 2
        
        # MLP predictions
        with torch.no_grad():
            mlp_pred = self.mlp_model(torch.tensor(X_full_scaled, dtype=torch.double).to(device)).cpu().numpy()
            if self.mixed:
                mlp_pred_flip = self.mlp_model(torch.tensor(X_full_flip_scaled, dtype=torch.double).to(device)).cpu().numpy()
                mlp_pred = (mlp_pred + mlp_pred_flip) / 2
        
        # LGBM predictions
        lgbm_pred = np.column_stack([m.predict(X_full_scaled) for m in self.lgbm_models])
        if self.mixed:
            lgbm_pred_flip = np.column_stack([m.predict(X_full_flip_scaled) for m in self.lgbm_models])
            lgbm_pred = (lgbm_pred + lgbm_pred_flip) / 2
        
        # Weighted ensemble
        ensemble_pred = self.gp_weight * gp_pred + self.mlp_weight * mlp_pred + self.lgbm_weight * lgbm_pred
        
        # Apply target-specific bias correction
        corrected_pred = ensemble_pred + self.corrections
        
        # Clip to valid range
        corrected_pred = np.clip(corrected_pred, 0, 1)
        
        return torch.tensor(corrected_pred)

print('TargetSpecificBiasEnsemble defined')

In [ ]:
# First, let's collect predictions and compute target-specific bias
print('=== Computing Target-Specific Bias ===')
X_single, Y_single = load_data("single_solvent")

# Collect predictions and actuals
all_preds = []
all_actuals = []

split_generator = generate_leave_one_out_splits(X_single, Y_single)
for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = TargetSpecificBiasEnsemble(data='single', corrections=np.array([0.0, 0.0, 0.0]))
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_preds.append(predictions.numpy())
    all_actuals.append(test_Y.values)

preds = np.vstack(all_preds)
actuals = np.vstack(all_actuals)

# Compute target-specific bias
mean_pred = preds.mean(axis=0)
mean_actual = actuals.mean(axis=0)
bias_per_target = mean_pred - mean_actual

print(f'\nMean predictions: {mean_pred}')
print(f'Mean actuals: {mean_actual}')
print(f'Bias per target: {bias_per_target}')
print(f'\nTarget-specific corrections (to add to predictions):')
print(f'  Product 2: {-bias_per_target[0]:.6f}')
print(f'  Product 3: {-bias_per_target[1]:.6f}')
print(f'  SM: {-bias_per_target[2]:.6f}')

In [ ]:
# Test different correction scales
print('\n=== Testing Different Correction Scales ===')

# The corrections should be the negative of the bias
base_corrections = -bias_per_target

scales = [0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5]
results = {}

for scale in scales:
    corrections = base_corrections * scale
    corrected_preds = np.clip(preds + corrections, 0, 1)
    mse = np.mean((corrected_preds - actuals) ** 2)
    results[scale] = mse
    print(f'Scale={scale:.2f}: Corrections={corrections}, MSE={mse:.6f}')

best_scale = min(results, key=results.get)
best_corrections = base_corrections * best_scale
print(f'\nBest scale: {best_scale} with MSE={results[best_scale]:.6f}')
print(f'Best corrections: {best_corrections}')
print(f'Baseline (scale=0): MSE={results[0.0]:.6f}')

In [ ]:
# Run full CV with best corrections
print(f'\n=== Full CV with Target-Specific Corrections (scale={best_scale}) ===')
print(f'Corrections: {best_corrections}')

X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = TargetSpecificBiasEnsemble(data='single', corrections=best_corrections)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')

In [ ]:
# Full data CV with best corrections
print(f'\n=== Full Data CV with Target-Specific Corrections (scale={best_scale}) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = TargetSpecificBiasEnsemble(data='full', corrections=best_corrections)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')

In [ ]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (Target-Specific Bias Correction, scale={best_scale}) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
    print(f'Predicted LB: {4.22 * overall_mse + 0.0534:.4f}')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

In [ ]:
# Create submission from CV predictions
submission_single = []
for fold_idx, preds in enumerate(all_predictions_single):
    for row_idx, row in enumerate(preds):
        submission_single.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
submission_single_solvent = pd.DataFrame(submission_single)

submission_full = []
for fold_idx, preds in enumerate(all_predictions_full):
    for row_idx, row in enumerate(preds):
        submission_full.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
submission_full_data = pd.DataFrame(submission_full)

# Combine and save
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

print(f'Submission saved: {len(submission)} rows')
print(f'Single solvent: {len(submission_single_solvent)}, Full data: {len(submission_full_data)}')

In [ ]:
# Final summary
print(f'\n=== FINAL SUMMARY ===')
print(f'\nModel: GP + MLP + LGBM Ensemble with Target-Specific Bias Correction')
print(f'Corrections: {best_corrections}')
print(f'Scale: {best_scale}')
print(f'Weights: GP={0.15}, MLP={0.55}, LGBM={0.30}')
print(f'\nCV Results:')
print(f'  Single Solvent MSE: {mse_single:.6f}')
print(f'  Full Data MSE: {mse_full:.6f}')
print(f'  Overall MSE: {overall_mse:.6f}')
print(f'\nBaseline (exp_032): CV 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')